# 프로젝트 : 모든 장르 간 편향성 측정해 보기

## 라이브러리 버전을 확인

In [3]:
import konlpy
import gensim
import sklearn
import seaborn

print(konlpy.__version__)
print(gensim.__version__)
print(sklearn.__version__)
print(seaborn.__version__)

In [5]:
from tqdm import tqdm, trange

In [4]:
import os
from konlpy.tag import Okt
from nltk.tokenize import sent_tokenize
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import pandas as pd

## STEP 1. 형태소 분석기를 이용하여 품사가 명사인 경우 해당 단어를 추출하기

In [6]:
genre_txt = ['synopsis_SF.txt', 'synopsis_family.txt', 'synopsis_show.txt', 'synopsis_horror.txt', 'synopsis_etc.txt', 
             'synopsis_documentary.txt', 'synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_musical.txt', 
             'synopsis_mystery.txt', 'synopsis_crime.txt', 'synopsis_historical.txt', 'synopsis_western.txt', 
             'synopsis_adult.txt', 'synopsis_thriller.txt', 'synopsis_animation.txt', 'synopsis_action.txt', 
             'synopsis_adventure.txt', 'synopsis_war.txt', 'synopsis_comedy.txt', 'synopsis_fantasy.txt']
genre_name = ['SF', '가족', '공연', '공포(호러)', '기타', '다큐멘터리', '드라마', '멜로로맨스', '뮤지컬', '미스터리', '범죄', '사극', '서부극(웨스턴)',
         '성인물(에로)', '스릴러', '애니메이션', '액션', '어드벤처', '전쟁', '코미디', '판타지']

In [8]:
import os

def read_synopsis(file_name):
    with open(os.getenv('HOME')+'/aiffel/weat/'+file_name, 'r') as file:
        for i in range(1):
            print(file.readline(), end='')

In [9]:
read_synopsis(genre_txt[0])

In [86]:
def read_files(file_list):
    texts = []
    for file_name in tqdm(file_list):
        with open(os.getenv('HOME')+'/aiffel/weat/'+file_name, 'r') as file:
            texts.append(file.read())
    return texts

In [87]:
texts = read_files(genre_txt)

In [12]:
from nltk.tokenize import sent_tokenize

In [13]:
import nltk
nltk.download('punkt')

In [14]:
len(texts)

In [37]:
def preprocess_texts_korean(texts):
    okt = Okt()
    processed_texts = []
    for text in tqdm(texts):
        genre_texts = []
        sentences = sent_tokenize(text)
        for sentence in sentences:
            words = []
            # 명사만 추출
            tokenlist = okt.pos(sentence, stem=True, norm=True)
            for word in tokenlist:
                if word[1] in ["Noun"]:
                    words.append(word[0])
        
            genre_texts.append(words)

        processed_texts.append(genre_texts)
    
    return processed_texts

In [42]:
processed_texts = preprocess_texts_korean(texts)

In [46]:
len(processed_texts)

In [49]:
"nouns_" + genre_txt[0]

In [51]:
for idx in trange(len(genre_txt)):
    file_name = "nouns_"+genre_txt[idx]

    with open(file_name, 'w+') as file:
        for num, i in enumerate(processed_texts[idx]):
            if num+1 < len(processed_texts[idx]):
                file.write(', '.join(i) + "\n")
            else:
                file.write(', '.join(i))

In [53]:
li = []
with open("nouns_" + genre_txt[0], "r") as file:
    for fi in file:
        ll = [ name.strip() for name in fi.split(",")]
        li.append(ll)
# li

## STEP 2. 추출된 결과로 embedding model 만들기

In [20]:
from gensim.models import Word2Vec

# tokenized에 담긴 데이터를 가지고 나만의 Word2Vec을 생성합니다. (Gensim 4.0 기준)
model = Word2Vec(processed_texts, vector_size=100, window=5, min_count=3, sg=0)  
model.wv.most_similar(positive=['영화'])

# Gensim 3.X 에서는 아래와 같이 생성합니다. 
# model = Word2Vec(tokenized, size=100, window=5, min_count=3, sg=0)  
# model.most_similar(positive=['영화'])

In [145]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('./w2v') 

In [146]:
model = KeyedVectors.load_word2vec_format("./w2v")
print("모델  load 완료!")

In [22]:
model.most_similar(positive=['사랑'])

In [23]:
model.wv.most_similar(positive=['연극'])

## STEP 3. target, attribute 단어 셋 만들기
----
이전 스텝에서는 TF-IDF를 사용해서 단어 셋을 만들었습니다. 이 방법으로도 어느 정도는 대표 단어를 잘 선정할 수 있습니다. 그러나 TF-IDF가 높은 단어를 골랐음에도 불구하고 중복되는 단어가 발생하는 문제가 있었습니다. 개념축을 표현하는 단어가 제대로 선정되지 않은 것은 WEAT 계산 결과에 악영향을 미칩니다.

TF-IDF를 적용했을 때의 문제점이 무엇인지 지적 가능하다면 그 문제점을 지적하고 스스로 방법을 개선하여 대표 단어 셋을 구축해 보기 바랍니다. TF-IDF 방식을 쓰더라도 중복된 단어를 잘 제거하면 여전히 유용한 방식이 될 수 있습니다.

## WEAT 구현하기

In [55]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [56]:
def cos_sim(i, j):
    return dot(i, j.T)/(norm(i)*norm(j))

def s(w, A, B):
    c_a = cos_sim(w, A)
    c_b = cos_sim(w, B)
    mean_A = np.mean(c_a, axis=-1)
    mean_B = np.mean(c_b, axis=-1)
    return mean_A - mean_B #, c_a, c_b

def weat_score(X, Y, A, B):
    
    s_X = s(X, A, B)
    s_Y = s(Y, A, B)

    mean_X = np.mean(s_X)
    mean_Y = np.mean(s_Y)
    
    std_dev = np.std(np.concatenate([s_X, s_Y], axis=0))
    
    return  (mean_X-mean_Y)/std_dev

## 3. TF-IDF로 해당 데이터를 가장 잘 표현하는 단어 셋 만들기

In [149]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from konlpy.tag import Okt

def read_token(file_name):
    okt = Okt()
    result = []
    with open(os.getenv('HOME')+'/aiffel/weat/'+file_name, 'r') as fread: 
        #print(file_name, '파일을 읽고 있습니다.')
        while True:
            line = fread.readline() 
            if not line: break 
            tokenlist = okt.pos(line, stem=True, norm=True) 
            for word in tokenlist:
                if word[1] in ["Noun"]:#, "Adjective", "Verb"]:
                    result.append((word[0])) 
    return ' '.join(result)

### art, gen

In [153]:
art_txt = 'synopsis_art.txt'
gen_txt = 'synopsis_gen.txt'

art = read_token(art_txt)
gen = read_token(gen_txt)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform([art, gen])

print(vectorizer.vocabulary_['영화'])
print(vectorizer.get_feature_names()[23976])

In [ ]:
m1 = X[0].tocoo()   # art를 TF-IDF로 표현한 sparse matrix를 가져옵니다. 
m2 = X[1].tocoo()   # gen을 TF-IDF로 표현한 sparse matrix를 가져옵니다. 

w1 = [[i, j] for i, j in zip(m1.col, m1.data)]
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True)   #art를 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 
w2.sort(key=lambda x: x[1], reverse=True)   #gen을 구성하는 단어들을 TF-IDF가 높은 순으로 정렬합니다. 

print('예술영화를 대표하는 단어들:')
for i in range(100):
    print(vectorizer.get_feature_names()[w1[i][0]], end=', ')

print('\n')
    
print('일반영화를 대표하는 단어들:')
for i in range(100):
    print(vectorizer.get_feature_names()[w2[i][0]], end=', ')

__<예술영화를 대표하는 단어들>__    
그녀, 자신, 시작, 위해, 사랑, 사람, 영화, 친구, 남자, 가족, 이야기, 마을, 사건, 마음, 세상, 아버지, 아이, 엄마, 모든, 여자, 대한, 서로, 과연, 다시, 시간, 아들, 소녀, 아내, 다른, 사이, 영화제, 세계, 사실, 하나, 점점, 남편, 감독, 여행, 인생, 발견, 모두, 순간, 우리, 가장, 마지막, 생활, 아빠, 모습, 통해, 죽음, 기억, 비밀, 학교, 음악, 한편, 소년, 생각, 도시, 명의, 사고, 결혼, 전쟁, 때문, 위기, 이제, 최고, 이자, 과거, 일상, 경찰, 상황, 간다, 미국, 결심, 운명, 현실, 관계, 지금, 단편, 여인, 하루, 이름, 이후, 준비, 인간, 감정, 만난, 국제, 처음, 충격, 살인, 누구, 동안, 존재, 그린, 어머니, 연인, 계속, 동생, 작품, 

__<일반영화를 대표하는 단어들>__    
자신, 그녀, 영화제, 위해, 사람, 시작, 국제, 영화, 친구, 사랑, 남자, 이야기, 대한, 서울, 여자, 사건, 남편, 아이, 가족, 아버지, 다른, 마을, 시간, 엄마, 아들, 모든, 단편, 마음, 사실, 다시, 세계, 모습, 작품, 통해, 생각, 서로, 세상, 발견, 감독, 아내, 관계, 소녀, 사이, 하나, 우리, 애니메이션, 때문, 여성, 죽음, 과연, 점점, 인간, 생활, 한편, 결혼, 상황, 모두, 기억, 명의, 소년, 여행, 가장, 간다, 순간, 이제, 도시, 비밀, 학교, 과거, 가지, 이자, 경찰, 마지막, 미국, 동안, 전쟁, 주인공, 대해, 존재, 현실, 연출, 사고, 살인, 일상, 어머니, 계속, 사회, 인생, 다큐멘터리, 부문, 섹스, 최고, 바로, 동생, 의도, 하루, 위기, 계획, 정체, 한국, 

두 개념을 대표하는 단어를 TF-IDF가 높은 순으로 추출하고 싶었는데, 양쪽에 중복된 단어가 너무 많은 것을 볼 수 있습니다.

두 개념축이 대조되도록 대표하는 단어 셋을 만들고 싶기 때문에 단어가 서로 중복되지 않게 단어셋을 추출해야 합니다. 우선 상위 100개의 단어들 중 중복되는 단어를 제외하고 상위 n(=15)개의 단어를 추출합니다.

In [ ]:
n = 15
w1_, w2_ = [], []
for i in range(100):
    w1_.append(vectorizer.get_feature_names()[w1[i][0]])
    w2_.append(vectorizer.get_feature_names()[w2[i][0]])

# w1에만 있고 w2에는 없는, 예술영화를 잘 대표하는 단어를 15개 추출한다.
target_art, target_gen = [], []
for i in range(100):
    if (w1_[i] not in w2_) and (w1_[i] in model.wv): target_art.append(w1_[i])
    if len(target_art) == n: break 

# w2에만 있고 w1에는 없는, 일반영화를 잘 대표하는 단어를 15개 추출한다.
for i in range(100):
    if (w2_[i] not in w1_) and (w2_[i] in model.wv): target_gen.append(w2_[i])
    if len(target_gen) == n: break

In [ ]:
print('예술영화를 대표하는 단어들:')
print(target_art)
print('\n')
print('일반영화를 대표하는 단어들:')
print(target_gen)

target_art = ['아빠', '음악', '결심', '운명', '지금', '여인', '이름', '이후', '준비', '감정', '만난', '처음', '충격', '누구', '그린']
     
     
target_gen = ['서울', '애니메이션', '여성', '가지', '주인공', '대해', '연출', '사회', '다큐멘터리', '부문', '섹스', '바로', '의도', '계획', '정체']

## 전체 장르 비교

In [19]:
genre_txt = ['synopsis_SF.txt', 'synopsis_family.txt', 'synopsis_show.txt', 'synopsis_horror.txt', 'synopsis_etc.txt', 
             'synopsis_documentary.txt', 'synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_musical.txt', 
             'synopsis_mystery.txt', 'synopsis_crime.txt', 'synopsis_historical.txt', 'synopsis_western.txt', 
             'synopsis_adult.txt', 'synopsis_thriller.txt', 'synopsis_animation.txt', 'synopsis_action.txt', 
             'synopsis_adventure.txt', 'synopsis_war.txt', 'synopsis_comedy.txt', 'synopsis_fantasy.txt']
genre_name = ['SF', '가족', '공연', '공포(호러)', '기타', '다큐멘터리', '드라마', '멜로로맨스', '뮤지컬', '미스터리', '범죄', '사극', '서부극(웨스턴)',
         '성인물(에로)', '스릴러', '애니메이션', '액션', '어드벤처', '전쟁', '코미디', '판타지']

In [41]:
genre = []
for file_name in tqdm(genre_txt):
    genre.append(read_token(file_name))

In [ ]:
len(genre)

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(genre)

print(X.shape)

In [37]:
m = [X[i].tocoo() for i in range(X.shape[0])]

w = [[[i, j] for i, j in zip(mm.col, mm.data)] for mm in m]

for i in range(len(w)):
    w[i].sort(key=lambda x: x[1], reverse=True)
attributes = []
for i in range(len(w)):
    print(genre_name[i], end=': ')
    attr = []
    j = 0
    while (len(attr) < 15):
        if vectorizer.get_feature_names()[w[i][j][0]] in model.wv:
            attr.append(vectorizer.get_feature_names()[w[i][j][0]])
            print(vectorizer.get_feature_names()[w[i][j][0]], end=', ')
        j += 1
    attributes.append(attr)
    print()

단순히 tf-idf로 구하게 되면 중복되는 단어셋이 많이 보인다    
중복단어 제거를 해줘야 한다

In [39]:
all_words = [text for texts in processed_texts for text in texts]
word_counts = Counter(all_words)

In [40]:
word_counts.most_common(5)

In [41]:
word_counts.most_common(30)

In [42]:
len(word_counts)

In [44]:
import pandas as pd

In [45]:
wc = pd.DataFrame({'word': word_counts.keys(),
             'cnt': word_counts.values()})

In [136]:
wc[wc.cnt>50].cnt.describe()

In [138]:
wc[wc.cnt>100]

In [50]:
wc = wc.sort_values(by='cnt',ascending=False)

In [76]:
import matplotlib.pyplot as plt
import numpy as np; 
import seaborn as sns; 

np.random.seed(0)
plt.figure(figsize=(15,4))
# 한글 지원 폰트
sns.set(font='NanumGothic')

fig.set_facecolor('white') #fig의 배경색

c = wc.head(40)

ax = sns.barplot(x=c['word'], y= c['cnt'])
ax.set_title('전체 단어 빈도')
plt.xticks(rotation=45) #xticks는 고객 명

plt.show()

In [80]:
wc.head(50)

In [78]:
sns.boxplot(wc.cnt)

In [230]:
# 공통 단어 리스트 생성
common_words = set([word for word, count in word_counts.items() if count > 1000])

In [231]:
# common_words

In [232]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(genre)

print(X.shape)


In [233]:
m = [X[i].tocoo() for i in range(X.shape[0])]
w = [[[i, j] for i, j in zip(mm.col, mm.data)] for mm in m]

In [234]:
for i in range(len(w)):
    w[i].sort(key=lambda x: x[1], reverse=True)

In [236]:
new_attributes = []
for i in range(len(w)):
    print(genre_name[i], end=': ')
    attr = []
    j = 0
    while (len(attr) <= 15):
        word = vectorizer.get_feature_names_out()[w[i][j][0]]
        if (word in model.wv) and (word not in common_words):
            attr.append(word)
            print(word, end=', ')
        j += 1
    new_attributes.append(attr)
    print()

## 4. embedding model과 단어 셋으로 WEAT score 구해보기

이제 WEAT_score를 구해봅시다.

traget_X는 art, target_Y는 gen, attribute_A는 '드라마', attribute_B는 '액션' 과 같이 정해줄 수 있습니다.

target_X 는 art, target_Y 는 gen으로 고정하고 attribute_A, attribute_B를 바꿔가면서 구해봅시다. 구한 결과를 21x21 매트릭스 형태로 표현해서 matrix 라는 변수에 담아봅시다.

In [237]:
from tqdm import trange

In [238]:
matrix = [[0 for _ in range(len(genre_name))] for _ in trange(len(genre_name))]

In [239]:
len(new_attributes)

In [240]:
X = np.array([model.wv[word] for word in target_art])
Y = np.array([model.wv[word] for word in target_gen])

for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        A = np.array([model.wv[word] for word in new_attributes[i]])
        B = np.array([model.wv[word] for word in new_attributes[j]])
        matrix[i][j] = weat_score(X, Y, A, B)

matrix를 채워보았습니다. WEAT score 값을 보고, 과연 우리의 직관과 비슷한지 살펴볼까요?

In [241]:
import pandas as pd

In [242]:
for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        print(genre_name[i], genre_name[j],matrix[i][j])

In [249]:
corr_df = pd.DataFrame(matrix, columns=genre_name, index=genre_name)

In [250]:
corr_df_unstack = pd.DataFrame(corr_df.unstack(), columns=['score']).sort_values(by='score')

In [251]:
corr_df_unstack.to_csv('corr_df_unstack.csv')

In [252]:
corr_df_unstack.reset_index(inplace=True)
corr_df_unstack.columns = ['A','B','score']

In [253]:
corr_df_unstack.to_csv('corr_df_unstack.csv', index=False)

## WEAT score가 0.8 이상, -0.8 이하의 경우

- 양수 점수: X(예술영화) 단어들이 A 속성 단어들과 더 많이 연관되어 있고, Y(일반영화) 단어들이 B 속성 단어들과 더 많이 연관되어 있음
- 음수 점수: X(예술영화) 단어들이 B 속성 단어들과 더 많이 연관되어 있고, Y(일반영화) 단어들이 A 속성 단어들과 더 많이 연관되어 있음
- 점수의 크기: 절대값이 클수록 편향의 정도가 큼

In [254]:
corr_df_unstack[np.abs(corr_df_unstack.score) >= 0.8]

In [245]:
high_corr = corr_df_unstack[np.abs(corr_df_unstack.score) >= 0.8]

In [246]:
high_corr.reset_index(inplace=True)
high_corr.columns = ['A','B','score']

### 예술 영화와 가까운 장르

In [255]:
x1 = high_corr[high_corr.score > 0].A.tolist()
x2 = high_corr[high_corr.score < 0].B.tolist()

print(x1)
print(x2)

### 일반영화와 가까운 장르

In [248]:
y1 = high_corr[high_corr.score < 0].A.tolist()
y2 = high_corr[high_corr.score > 0].B.tolist()

print(y1)
print(y2)

### 결과 해석
- weat score에 따른 결과를 해석해보면 예술 영화와 가까운 장르는 다큐멘터리, SF, 공연, 뮤지컬이 나타났고 이는 우리의 상식에 부합한 결과가 나왔다
- 일반 영화와 가까운 장르는 판타지, 코미디, 멜로로멘스, 드라마, 가족 등 일반적인 우리의 생각과 유사한 결과가 나왔다

## STEP 4. WEAT score 계산과 시각화
---
영화 구분, 영화 장르에 따른 편향성을 측정하여 WEAT score로 계산해 보고 이를 Heatmap 형태로 시각화해 봅시다. 편향성이 두드러지는 영화장르 attribute 구성에는 어떤 케이스가 있는지 시각적으로 두드러지게 구성되면 좋습니다.

In [256]:
import matplotlib.pyplot as plt
import numpy as np; 
import seaborn as sns; 

np.random.seed(0)
plt.figure(figsize=(15,8))
# 한글 지원 폰트
sns.set(font='NanumGothic')

# 마이너스 부호 
plt.rcParams['axes.unicode_minus'] = False

ax = sns.heatmap(matrix, xticklabels=genre_name, yticklabels=genre_name, annot=True,  cmap='RdYlGn_r')
ax

# 더 좋은 키워드를 뽑아 내기 위한 시도

## Bertopic
- 결과적으로 먼저 말하자면 좋은 토픽을 뽑아내지 못했다.
- 이는 시놉시스를 바탕으로 키워드를 뽑아내서 줄거리의 내용들을 바탕으로 키워드를 뽑아서 잘 안된 것 같다
- 오히려 장르별 리뷰 데이터가 있다면 더 좋은 결과가 나오지 않을까 싶다 

In [54]:
# !pip install transformers
# !pip install bertopic

In [3]:
from transformers import AutoTokenizer, AutoModel
from bertopic import BERTopic
import torch

In [7]:
def get_bertopic_keywords(data):
    num_topics = 2

    topic_model = BERTopic(language='korean', nr_topics=num_topics)
    topics, _ = topic_model.fit_transform(data)

    #topic_info = topic_model.get_topic_info()
    representative_keywords = topic_model.get_topic(0)

    return representative_keywords

### 위에서 저장해둔 processed_text 불러오기

In [10]:
processed_texts = []

for txt in tqdm(genre_txt):
    li = []
    with open("nouns_" + txt, "r") as file:
        for fi in file:
            ll = [ name.strip() for name in fi.split(",")]
            li.append(ll)
    processed_texts.append(li)

In [11]:
len(processed_texts)

In [12]:
keywords = {}

for i in trange(len(processed_texts)):
    data = [' '.join(texts) for texts in processed_texts[i]]
    representative_keywords = get_bertopic_keywords(data)
    keywords[genre_name[i]] = representative_keywords
    print(f'{genre_name[i]} : ', representative_keywords)

In [15]:
keywords[genre_name[1]]

- 좋은 키워드가 잘 뽑히지 않고 시간도 너무 오래걸려서 사용하지 않기로 결정했다

## krwordrank

In [89]:
# krwordrank 설치

!pip install krwordrank

In [22]:
# library import

import pandas as pd
import numpy as np
from krwordrank.word import KRWordRank
from krwordrank.word import summarize_with_keywords
from krwordrank.sentence import summarize_with_sentences
import pickle

In [23]:
def get_krwordrank(texts, min_count, max_length, beta, maxiter):

    a = [' '.join(text) for text in texts]
    
    # Keyword 알고리즘에 데이터 적용
    keywords = summarize_with_keywords(a, min_count=min_count, max_length=max_length, beta=beta, max_iter=maxiter, verbose=True)
    
    # 키워드 뽑아내기.
    for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
       print(word, end=', ')
        
    return keywords

In [25]:
# 하이퍼파라미터
min_count=2 
max_length=100
beta=0.85
max_iter=15

# keywords = get_krwordrank(processed_texts[0], min_count, max_length, beta, max_iter)

In [34]:
kr_keywords = {}

for i in range(len(processed_texts)):
    keywords = get_krwordrank(processed_texts[i], min_count, max_length, beta, max_iter)
    kr_keywords[genre_name[i]] = keywords

In [35]:
genre_keywords = {}

for k in kr_keywords.keys():
    print(k)
    words = []
    for word, r in sorted(kr_keywords[k].items(), key=lambda x:x[1], reverse=True)[:30]:
        words.append(word)
    print(words)
    print()
    genre_keywords[k] = words

- 위의 방식으로 단순하게 키워드를 뽑아내니 겹치는 단어들이 자주 등장하였다
- 전체 데이터셋에서 키워드를 뽑아내서 상위 키워드를 common_keywords에 넣어둔뒤 이를 불용어 처리하여 키워드에서 제거해 줄 것이다

### 모든 장르의 데이터를 합친 뒤 키워드 추출

In [27]:
all_texts = [text for texts in processed_texts for text in texts]
all_keywords = get_krwordrank(all_texts, min_count, max_length, beta, max_iter)

In [30]:
common_keywords = []

for word, r in sorted(all_keywords.items(), key=lambda x:x[1], reverse=True)[:50]:
    common_keywords.append(word)
    print(word, r)

In [31]:
' '.join(common_keywords)

In [32]:
common_keywords.extend(['충무','몰래','한편','이자','사실','어머','만난'])

- 추가적으로 직관에 맞지않는 단어들을 제거해 주었다

In [39]:
genre_keywords_stopwords = {}

for k in kr_keywords.keys():
    print(k)
    words = []
    for word, r in sorted(kr_keywords[k].items(), key=lambda x:x[1], reverse=True):
        if word not in common_keywords:
            words.append(word)
    print(words[:30])
    print()
    genre_keywords[k] = words[:30]

In [40]:
genre_keywords

In [41]:
pd.DataFrame(genre_keywords).T.to_csv('kr_keywords.csv')

## weat score 다시 측정

target_art = ['아빠', '음악', '결심', '운명', '지금', '여인', '이름', '이후', '준비', '감정', '만난', '처음', '충격', '누구', '그린']
     
     
target_gen = ['서울', '애니메이션', '여성', '가지', '주인공', '대해', '연출', '사회', '다큐멘터리', '부문', '섹스', '바로', '의도', '계획', '정체']

- 예술 영화를 대표하는 단어와 일반영화를 대표하는 단어 자체도 이미 장르에 편향이 되어있는 것으로 보아 다시 키워드 추출부터 하고자 한다

### 예술영화와 일반영화 키워드 추출

In [17]:
art_txt = 'synopsis_art.txt'
gen_txt = 'synopsis_gen.txt'

art_gen_txt = [art_txt, gen_txt]
art_gen_txt 

In [272]:
texts = read_files(art_gen_txt)

In [274]:
# art_gen_texts = preprocess_texts_korean(texts)

In [275]:
len(art_gen_texts)

In [175]:
for idx in trange(len(art_gen_txt)):
    file_name = "nouns_"+art_gen_txt[idx]

    with open(file_name, 'w+') as file:
        for num, i in enumerate(art_gen_texts[idx]):
            if num+1 < len(art_gen_texts[idx]):
                file.write(', '.join(i) + "\n")
            else:
                file.write(', '.join(i))

## 데이터셋 다시 불러오기

In [18]:
art_gen_texts = []

for txt in tqdm(art_gen_txt):
    li = []
    with open("nouns_" + txt, "r") as file:
        for fi in file:
            ll = [ name.strip() for name in fi.split(",")]
            li.append(ll)
    art_gen_texts.append(li)

In [19]:
len(art_gen_texts)

In [42]:
art_gen_keywords = {}
name = ['art','gen']

for i in range(len(art_gen_texts)):
    keywords = get_krwordrank(art_gen_texts[i], min_count, max_length, beta, max_iter)
    print(name[i])
    words = []
    for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True):
        if word not in common_keywords:
            words.append(word)
    print(words[:30])
    print()
    art_gen_keywords[name[i]] = words

### 겹치는 단어들을 제거하고 상위 15개 단어를 선정했다

In [43]:
art_gen_list = list(art_gen_keywords.values())

In [44]:
art = art_gen_list[0]
gen = art_gen_list[1]

art = [x for x in art if x not in gen[:30]]
gen = [x for x in gen if x not in art[:30]]

target_art = art[:15]
target_gen = gen[:15]

In [45]:
print('예술 영화의 주요 키워드 :', ' '.join(target_art))
print('일반 영화의 주요 키워드 :', ' '.join(target_gen))

### weat score 다시 계산하기

In [46]:
matrix = [[0 for _ in range(len(genre_name))] for _ in trange(len(genre_name))]

In [47]:
attrs = list(genre_keywords.values())

In [48]:
len(attrs)

In [50]:
from gensim.models import KeyedVectors

In [51]:
model = KeyedVectors.load_word2vec_format("./w2v")
print("모델  load 완료!")

In [57]:
X = np.array([model[word] for word in target_art])
Y = np.array([model[word] for word in target_gen])

for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        A = np.array([model[word] for word in attrs[i] if word in model])
        B = np.array([model[word] for word in attrs[j] if word in model])
        matrix[i][j] = weat_score(X, Y, A, B)

matrix를 채워보았습니다. WEAT score 값을 보고, 과연 우리의 직관과 비슷한지 살펴볼까요?

In [58]:
import pandas as pd

In [59]:
for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        print(genre_name[i], genre_name[j],matrix[i][j])

In [60]:
corr_df = pd.DataFrame(matrix, columns=genre_name, index=genre_name)

In [66]:
corr_df_unstack = pd.DataFrame(corr_df.unstack(), columns=['score']).sort_values(by='score')

In [67]:
corr_df_unstack.reset_index(inplace=True)
corr_df_unstack.columns = ['A','B','score']

In [68]:
corr_df_unstack.to_csv('new_corr_df_unstack.csv', index=False)

In [69]:
corr_df_unstack[np.abs(corr_df_unstack.score) >= 0.6]

In [78]:
high_corr = corr_df_unstack[np.abs(corr_df_unstack.score) >= 0.7]

In [79]:
high_corr

### 예술 영화와 가까운 장르

In [80]:
x1 = high_corr[high_corr.score > 0].A.tolist()
x2 = high_corr[high_corr.score < 0].B.tolist()

print(x1)
print(x2)

### 일반영화와 가까운 장르

In [81]:
y1 = high_corr[high_corr.score < 0].A.tolist()
y2 = high_corr[high_corr.score > 0].B.tolist()

print(y1)
print(y2)

- 예술 영화에 대한 키워드가 아쉬웠는지 직관과 유사한 결과가 나오지 않아서 아쉽다

## STEP 4. WEAT score 계산과 시각화
---
영화 구분, 영화 장르에 따른 편향성을 측정하여 WEAT score로 계산해 보고 이를 Heatmap 형태로 시각화해 봅시다. 편향성이 두드러지는 영화장르 attribute 구성에는 어떤 케이스가 있는지 시각적으로 두드러지게 구성되면 좋습니다.

In [82]:
import matplotlib.pyplot as plt
import numpy as np; 
import seaborn as sns; 

np.random.seed(0)
plt.figure(figsize=(15,8))
# 한글 지원 폰트
sns.set(font='NanumGothic')

# 마이너스 부호 
plt.rcParams['axes.unicode_minus'] = False

ax = sns.heatmap(matrix, xticklabels=genre_name, yticklabels=genre_name, annot=True,  cmap='RdYlGn_r')
ax

## KeyBert

In [98]:
# !pip install keybert

In [97]:
from keybert import KeyBERT

doc = texts[0]
# kw_model = KeyBERT()
# keywords = kw_model.extract_keywords(doc)

In [91]:
' '.join(common_keywords)

In [93]:
from keybert import KeyBERT
from transformers import BertModel

In [94]:
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

In [112]:
input_texts = [[' '.join(text) for text in texts] for texts in processed_texts]

In [113]:
len(input_texts)

In [115]:
doc = ' '.join(input_texts[0])

In [120]:
genre_name[0]

In [121]:
keybert_kw = {}

for i in trange(len(input_texts)):
    kw = kw_model.extract_keywords(' '.join(input_texts[i]), keyphrase_ngram_range=(1, 1), top_n=20)
    keybert_kw[genre_name[i]] = kw
    
    print(genre_name[i])
    for k, _ in kw:
        print(k, end=' ')
        
    print()

- 생각해보니 이 데이터는 하나의 시놉시스를 담고 있기 때문에 그 시놉시스에 specific하게 키워드가 추출되는 것 같다
- 더 많은 데이터가 있다면 더 좋은 키워드가 뽑힐 수 있지 않을까 싶다
- keyBert는 인물 위주로 많이 뽑히는 것 같다 왜그럴까?

In [127]:
for key in keybert_kw :
    val = keybert_kw[key]
    print(f'{key} :', ', '.join([v[0] for v in val]))
    print()

## 전체 데이터 비지도학습으로 분리 (LDA)

In [156]:
docs = [' '.join(t) for text in processed_texts for t in text]

In [133]:
# 상위 1,000개의 단어를 보존 
vectorizer = TfidfVectorizer(max_features= 1000)
X = vectorizer.fit_transform(docs)

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :', X.shape)

In [135]:
from sklearn.decomposition import LatentDirichletAllocation

In [157]:
lda_model = LatentDirichletAllocation(n_components=21,learning_method='online',random_state=777, max_iter=1)

In [158]:
lda_top = lda_model.fit_transform(X)

In [159]:
print(lda_model.components_)
print(lda_model.components_.shape)

In [160]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [feature_names[i] for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_, terms, n=10)

In [164]:
# !pip install pyLDAvis

In [193]:
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim

In [204]:
import gensim

all_tokens = [t for text in processed_texts for t in text]
dictionary = gensim.corpora.Dictionary(all_tokens)
# 출현빈도가 적거나 자주 등장하는 단어는 제거
dictionary.filter_extremes(no_below=10, no_above=0.7)
print('Number of unique tokens: %d' % len(dictionary))

corpus = [dictionary.doc2bow(text) for text in all_tokens]

In [205]:
num_topics = 21

In [206]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)

In [207]:
# Get the topics
topics = lda_model.show_topics(num_topics=num_topics, num_words=10, log=False, formatted=False)

# Print the topics
for topic_id, topic in topics:
    print("Topic: {}".format(topic_id))
    print("Words: {}".format([word for word, _ in topic]))

In [215]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns

# 한글 지원 폰트
sns.set(font='NanumGothic')

# 워드클라우드를 그릴 수 있는 레이아웃 크기 설정
num_rows = 5
num_cols = 5

# Figure 크기 설정
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 20))

# 각 subplot에 워드클라우드 그리기
for topic_id, topic in enumerate(lda_model.print_topics(num_topics=num_topics, num_words=20)):
    topic_words = " ".join([word.split("*")[1].strip() for word in topic[1].split(" + ")])
    wordcloud = WordCloud(width=800, height=800, random_state=21, max_font_size=110, font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf').generate(topic_words)
    
    ax = axes[topic_id // num_cols, topic_id % num_cols]
    ax.imshow(wordcloud, interpolation="bilinear")
    ax.axis("off")
    ax.set_title("Topic: {}".format(topic_id), fontsize=15)

# 빈 subplot 처리
for i in range(len(lda_model.print_topics(num_topics=num_topics, num_words=20)), num_rows * num_cols):
    ax = axes[i // num_cols, i % num_cols]
    ax.axis("off")

plt.tight_layout(pad=0)
plt.show()


- topic 7 : 가족영화처럼 보인다
- topic 11, 18 : 애니메이션, 영화제, 부천 판타스틱 영화제 등 예술 영화 관련 내용들로 보인다

## 예술영화와 일반영화

In [131]:
art_gen_docs = [' '.join(t) for text in art_gen_texts for t in text]

In [148]:
# 상위 1,000개의 단어를 보존 
ag_vectorizer = TfidfVectorizer(max_features= 1000)
ag_X = ag_vectorizer.fit_transform(art_gen_docs)

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :', ag_X.shape)

In [149]:
lda_model = LatentDirichletAllocation(n_components=2,learning_method='online',random_state=777, max_iter=1)

In [150]:
lda_top = lda_model.fit_transform(ag_X)

In [151]:
print(lda_model.components_)
print(lda_model.components_.shape)

In [152]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = ag_vectorizer.get_feature_names()
get_topics(lda_model.components_, terms, n=10)

## 결론
- 결론적으로 이번 task는 시놉시스도 부족하고 일반영화와 예술영화의 label의 문제도 있을 것이고 데이터도 단어의 양적인 측면에서 일반영화의 데이터가 훨씬 많고, 예술영화가 그에 비해 반도 안되는 용량이였기에 데이터도 불균형 한 문제가 있다
- tf-idf의 결과가 우리의 직관과 유사한 결론이 나왔지만 이 결과도 사실 이미 일반 영화의 키워드에 '다큐멘터리','섹스'등 특정 장르를 지칭하는 단어들이 존재하였고, 이에 따라 나온 결과로 볼 수 있다
- 더 많은 데이터셋으로 장르를 대표할 수 있을 키워드를 찾는 것이 더 합당한 결론이 나올 것으로 보인다

# 회고
- 배운 점 
    - 기존에 배웠던 토픽 모델링을 적용해보고 나아가 bertopic도 해봤다
- 아쉬운 점 
    - 좀 더 체계적으로 비교를 해보고 싶었는데 시간상의 이유로 제대로 비교해보지 못해서 아쉽다
    - 생각보다 좋은 결과가 나오지 않아서 아쉽다
- 느낀 점
    - 시간이 너무 오래걸리고 내 가정과 비슷한 결론을 가져가는 것은 쉽지 않다
    - 더 많은 데이터가 필요하다
- 어려웠던 점 
    - 커널이 죽지 않게 잘 관리하는 것이 어렵다